In [1]:
    from rites import update_hooks, NotebookLoader
    
    from IPython import get_ipython
    from nbconvert.exporters.templateexporter import TemplateExporter
    from IPython.core.inputtransformer import InputTransformer
    from jinja2 import Environment
    from collections import UserList
    from dataclasses import dataclass, field

    @dataclass
    class Template(InputTransformer, UserList):
        environment = Environment()
        user_ns: dict = field(default=None, repr=False)
        data: list = field(default_factory=list)
        
        def __post_init__(self): 
            if self.user_ns is None:
                self.user_ns = get_ipython().user_ns
            transforms = get_ipython().input_transformer_manager.python_line_transforms
            self in transforms or transforms.insert(0, self)
            
        def push(self, line): self.data.append(line)
        
        def reset(self, *, str=""""""):
            while self.data: str += self.data.pop(0) + "\n"
            return self.environment.from_string(str).render(**self.user_ns)
        
        def unload(self, *args): 
            get_ipython().input_transformer_manager.python_line_transforms = list(
                filter(self.__ne__, get_ipython().input_transformer_manager.python_line_transforms))

In [2]:
    class TemplateLoader(NotebookLoader):
        def exec_module(Notebook, module): 
            template = Template(user_ns=module.__dict__)
            module = super().exec_module(module)
            template.unload()
            return module

In [3]:
    def load_ipython_extension(ip=get_ipython()):
        global TRANSFORMER
        TRANSFORMER = Template()
        update_hooks()
        update_hooks(TemplateLoader)
        
    def unload_ipython_extension(ip=None):
        update_hooks()
        global TRANSFORMER
        try:
            TRANSFORMER.unload()
        except: ...

In [4]:
    if __name__ == '__main__':
        load_ipython_extension()
        import testing